In [1]:
import pandas as pd
import numpy as np

import os
import glob

In [13]:
d = {'mgcr_inst_gen_ctrct_number': 'str', 'mejoravit_mtge_prdt_mark_type': 'str'}

In [14]:
df = pd.read_csv('t_pdm_mortgage_20220125.csv', dtype=d)

C:\Users\XMX2212_1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,15,16,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
df.dtypes

mtge_info_orig_source_name        object
mgcr_request_register_date        object
entity_name                       object
state_name                        object
branch_division_name              object
mortgage_destination_name         object
MONTO                            float64
mgcr_inst_gen_ctrct_number        object
mejoravit_mtge_prdt_mark_type     object
mgcr_own_comm_network_type        object
mgcr_balance_goal_amount         float64
mgcr_operation_goal_number       float64
mgcr_sum_total_amount            float64
mgcr_oper_sum_total_number       float64
mgcr_weighted_tax_amount         float64
contract_register_branch_name     object
mortgage_line_type                object
mgcr_limit_range_desc             object
mortgage_total_balance_amount    float64
ppty_township_name                object
dtype: object

In [18]:
df

,mtge_info_orig_source_name,mgcr_request_register_date,entity_name,state_name,branch_division_name,mortgage_destination_name,MONTO,mgcr_inst_gen_ctrct_number,mejoravit_mtge_prdt_mark_type,mgcr_own_comm_network_type,mgcr_balance_goal_amount,mgcr_operation_goal_number,mgcr_sum_total_amount,mgcr_oper_sum_total_number,mgcr_weighted_tax_amount,contract_register_branch_name,mortgage_line_type,mgcr_limit_range_desc,mortgage_total_balance_amount,ppty_township_name
0,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entre 25 y 50 %,NaN,NaN
1,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mas de 50 y hasta 60%,NaN,NaN
2,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mas de 80 y hasta 90%,NaN,NaN
3,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,8. Creditos Hipotecarios (incluye destinos del...,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entre 25 y 50 %,NaN,NaN
4,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,8. Creditos Hipotecarios (incluye destinos del...,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mas de 50 y hasta 60%,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313074,Mercado,01/12/2021,BBVA,Nuevo Leon,Noreste,5. Credito para Liquidez,3.503250,1,0,Patrimonial,NaN,NaN,NaN,NaN,346821.750,Monterrey Valle Sena,NaN,NaN,NaN,NaN
313075,Mercado,01/12/2021,BBVA,Ciudad De Mexico,Metropolitana,2. Adquisicion de vivienda,5.122500,5,0,Hipotecaria,NaN,NaN,NaN,NaN,468512.500,Punto MA Quevedo,NaN,NaN,NaN,NaN
313076,Mercado,01/12/2021,BBVA,Ciudad De Mexico,Metropolitana,4. Pago de Pasivos Hipotecarios,0.580000,1,0,Hipotecaria,NaN,NaN,NaN,NaN,46400.000,Punto MA Quevedo,NaN,NaN,NaN,NaN
313077,Mercado,01/12/2021,BBVA,Nuevo Leon,Noreste,2. Adquisicion de vivienda,6.095000,2,0,Patrimonial,NaN,NaN,NaN,NaN,478457.500,San Nicolas Anahuac,NaN,NaN,NaN,NaN


In [19]:
encoding = 'utf-8-sig'
df.to_csv(f't_pdm_mortgage_20220125_{encoding}.csv', index=False, encoding=encoding)

In [20]:
cols = df.columns
cols

Index(['mtge_info_orig_source_name', 'mgcr_request_register_date',
       'entity_name', 'state_name', 'branch_division_name',
       'mortgage_destination_name', 'MONTO', 'mgcr_inst_gen_ctrct_number',
       'mejoravit_mtge_prdt_mark_type', 'mgcr_own_comm_network_type',
       'mgcr_balance_goal_amount', 'mgcr_operation_goal_number',
       'mgcr_sum_total_amount', 'mgcr_oper_sum_total_number',
       'mgcr_weighted_tax_amount', 'contract_register_branch_name',
       'mortgage_line_type', 'mgcr_limit_range_desc',
       'mortgage_total_balance_amount', 'ppty_township_name'],
      dtype='object')

In [21]:
int_cols = list(filter(lambda s: 'number' in s, cols))


In [ ]:
# df = pd.read_csv('t_pdm_mortgage_20220125.csv', dtype=)